In [46]:
import argparse
from functools import partial
from pathlib import Path
import pandas as pd
from operator import itemgetter
import os
import math
import numpy as np
from random import shuffle
from time import time
from NTK import kernel_value, kernel_value_batch
from resampling import NestedCV, BaseModel
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.svm import SVC

In [2]:
DEFAULT_DATASET_PATH = Path("/data/pfizer_tx/tasks_all_clr/all_clr_train_LUAD_stage.h5")
keys = ['/expression', '/labels']
test_data = {key : pd.read_hdf(DEFAULT_DATASET_PATH, key = key) for key in keys}

In [48]:
X_tx = test_data['/expression']
Y_tx = test_data['/labels']
X_tx.shape, Y_tx.shape

((542, 57992), (542,))

In [4]:
MAX_DEPTH = 5 
C_LIST = [10.0 ** i for i in range(-2, 5)] # hyperparameter for NTK
n_classes = len(set(Y_tx)) # n classes
n_features = X_tx.shape[1] # n features

In [5]:
def alg(K_train, K_val, y_train, y_val, C):
    clf = SVC(kernel = "precomputed", C = C, cache_size = 100000, probability=True)
    clf.fit(K_train, y_train)
    y_hat = clf.predict_proba(K_val)[:,1]
    return roc_auc_score(y_val, y_hat)

In [6]:
# calculate NTK
Ks = kernel_value_batch(X_tx, MAX_DEPTH)

In [7]:
idxs = [e for e in range(len(Y_tx))]
shuffle(idxs)
train_fold, val_fold = idxs[:350], idxs[350:]
y = Y_tx

I think train_fold and val_fold are indices for these folds of data

In [8]:
# load training and validation set
cv_results = dict()

In [15]:
# enumerate kernels and cost values to find the best hyperparameters
for depth in range(MAX_DEPTH):
    for fix_depth in range(depth + 1):
        K = Ks[depth][fix_depth]
        for c in C_LIST:
            auc = alg(
                K_train=K[train_fold][:, train_fold], 
                K_val=K[val_fold][:, train_fold], 
                y_train=y[train_fold], 
                y_val=y[val_fold], 
                C=c)
            key_ = f"{depth},{fix_depth},{c}"
            try:
                cv_results[key_].append(auc)
            except KeyError:
                cv_results[key_] = [auc]

In [22]:
cv_results;
mean_results = {k : np.mean(v) for k, v in cv_results.items()}

In [29]:
best_params = max(mean_results.items(), key=itemgetter(1))[0]
best_params

'4,4,100.0'

In [109]:
class NTKNestedCV:
    def __init__(self, alg, hparams, n_splits_outer=5, n_splits_inner=5, verbose=False):
        """Nested cross validation class for NTK models only. 
        Currently only uses AUROC as its performance metric.
        
        Parameters
        ----------
        alg : func
                A function the implements a model that can accept the NTK kernel.  
                Currently only SVM works.  
                Model selection will be made based on the performance metric that 'alg' returns.  
                At present, this is roc_auc_score.  
        hparams : dict
                A dict containing all the hyperparameters to be tried. 
                Must contain ['C'] (the list of values for the SVM model)  
                and ['max_depth' for the NT kernel.]
        n_splits_outer : int
                The number of splits in the outer loop
        n_splits_inner : int
                The number of splits in the inner loop
        verbose : bool
                If True, prints all the parameter values being tested.
                        
        """
        self.alg = alg
        self.hparams = hparams
        self.n_splits_outer = n_splits_outer
        self.n_splits_inner = n_splits_inner
        self.verbose = verbose
        
    def _inner_loop(self, input_x, input_y):
        """The inner x-fold CV loop for finding optimal hyperparameters"""
        inner = KFold(n_splits=self.n_splits_inner)
        # instead of an array, we use a dict where the keys are "<depth>,<fixed_depth>,<C>" 
        # and the values are lists to which we can append
        inner_fold_results = dict()
        Ks = kernel_value_batch(input_x, self.hparams['max_depth'])
        
        for split_idx, (t, v) in enumerate(inner.split(input_x)):
            print(f"Inner fold {split_idx+1} of {self.n_splits_inner}")
            for depth in range(MAX_DEPTH):
                for fix_depth in range(depth + 1):
                    K = Ks[depth][fix_depth]
                    for c in C_LIST:
                        if self.verbose:
                            print(f"Fitting model with depth: {depth}, fix depth: {fix_depth}, C: {c}")
                        auc = self.alg(
                            K_train=K[t][:, t], 
                            K_val=K[v][:, t], 
                            y_train=y[t], 
                            y_val=y[v], 
                            C=c)
                        key_ = f"{depth},{fix_depth},{c}"
                        try:
                            inner_fold_results[key_].append(auc)
                        except KeyError:
                            inner_fold_results[key_] = [auc]

        # which hparam combination has best average performance across all splits?
        # select these params, train on all the data and return a trained model
        mean_results = {k : np.mean(v) for k, v in inner_fold_results.items()}
        best_params = max(mean_results.items(), key=itemgetter(1))[0]
        best_depth, best_fix, best_C = best_params.split(',')
        print(f"Best params: depth = {best_depth}, fixed depth = {best_fix}, C = {best_C}")
        
        return {"best_depth" : int(best_depth), "best_fix" : int(best_fix), "best_C" : float(best_C)}
    
    def _outer_loop(self, X, Y):
        """The outer loop that produces an unbiased estimate of large sample performance"""
        start = time()
        outer = KFold(n_splits=self.n_splits_outer)
        outer_fold_results = dict()
        for split_idx, (t, v) in enumerate(outer.split(X)):
            print(f"Outer fold {split_idx+1} of {self.n_splits_outer}")
            x_train, y_train = X.iloc[t], Y.iloc[t]
            x_test, y_test = X.iloc[v], Y.iloc[v]
            best_params = self._inner_loop(x_train.values, y_train.values)
            Ks = kernel_value_batch(X.values, best_params['best_depth']+1)
            K = Ks[int(best_params['best_depth'])][int(best_params['best_fix'])]
            this_performance = self.alg(
                                K_train=K[t][:, t], 
                                K_val=K[v][:, t], 
                                y_train=y[t], 
                                y_val=y[v], 
                                C=best_params['best_C'])
            outer_fold_results[f"Fold {split_idx+1}"] = (best_params, this_performance)
        time_taken = time() - start
        return (outer_fold_results, time_taken)
    
    def train(self, X, Y):
        """The main train loop for the class. 
        Takes the full input and output data and returns an array of performance measures (one for each outer loop) and the coresponding model paramaeters
        """
        outer_fold_results, time_taken = self._outer_loop(X, Y)
        mean_performance = np.mean([v for p, v in outer_fold_results.values()])
        print(f"Total time taken: {time_taken}")
        print(f"Mean performance across {self.n_splits_outer} outer splits: {mean_performance}")
        return outer_fold_results

In [110]:
ntk_nest_cv = NTKNestedCV(alg=alg, hparams={'max_depth' : 5, 'C' : C_LIST}, n_splits_outer=2, n_splits_inner=2)

In [111]:
results = ntk_nest_cv.train(X_tx, Y_tx)

Outer fold 1 of 2
Inner fold 1 of 2
Inner fold 2 of 2
Best params: depth = 4, fixed depth = 1, C = 0.01
Outer fold 2 of 2
Inner fold 1 of 2
Inner fold 2 of 2
Best params: depth = 4, fixed depth = 0, C = 100.0
Total time taken: 5.9234840869903564
Mean performance across 2 outer splits: 0.633664282577326


In [108]:
[v for p, v in results.values()]

[0.6129720853858784, 0.6539243365330322]

In [90]:
class NTK(BaseModel):
    def __init__(self, params):
        super().__init__()
        self.params = params
    def fit(self, X, y):
        Ks = kernel_value_batch(X, self.params['max_depth'])
        self.best_auc = 0.0
        self.best_depth = 0
        self.best_fix = 0
        self.clf = SVC(kernel = "precomputed", C = self.params['C'], cache_size = 100000, probability=True)
        for depth in range(self.params['max_depth']):
            for fix_depth in range(depth + 1):
                K = Ks[depth][fix_depth]
                self.clf.fit(K, y)
                y_hat = self.clf.predict_proba(K)[:,1]
                auc = roc_auc_score(y, y_hat)
                if auc > best_auc:
                    self.best_auc = auc
                    self.best_depth = depth
                    self.best_fix = fix_depth
        # fit the best model
        print ("Best AUC:", self.best_auc, "\tDepth:", self.best_depth, "\tFix:", self.best_fix)
        K = Ks[self.best_depth][self.best_fix]
        self.clf.fit(K, y)

    def predict_proba(self,X):
        Ks = kernel_value_batch(X, self.params['max_depth'])
        K = Ks[self.best_depth][self.best_fix]
        y_hat = self.clf.predict_proba(K)[:,1]
        return y_hat

In [91]:
params = {"max_depth" : 5, "C" : 10}

In [92]:
model = NTK(params)

In [93]:
model.fit(X_tx, Y_tx)

Best AUC: 1.0 	Depth: 4 	Fix: 4


In [94]:
y_hat = model.predict_proba(X_tx)

In [95]:
roc_auc_score(Y_tx, y_hat)

1.0